# Synthetic Chest X-Ray Image Generation with PGGAN
In this notebook, we will demonstrate the functionality of ``Scheduler`` which enable advanced training schemes such as progressive training as described in [PGGAN paper] (https://arxiv.org/pdf/1710.10196.pdf). We will train a PGGAN to produce high resolution synthetic frontal chest X-ray images where both the generator and the discriminator grows from $4 \times 4$ to $1024 \times 1024$.

In [ ]:
import os
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend

import fastestimator as fe

## Defining Input Pipeline

In [ ]:
from fastestimator.dataset.nih_chestxray import load_data
from fastestimator.op import TensorOp

from fastestimator.op.numpyop import ImageReader
from fastestimator.op.numpyop import Resize as ResizeRecord
from fastestimator.util.record_writer import RecordWriter

## Defining Network

In [ ]:
from fastestimator.architecture.pggan import build_G, build_D
from fastestimator.op.tensorop import Loss, ModelOp, Reshape, Resize
from fastestimator.schedule import Scheduler


## Defining Trace

In [ ]:
from fastestimator.trace import Trace

## Defining Estimator